# En construcción...

# Estimación de abundancia y densidad mediante métodos de captura-marcado-recaptura

In [9]:
# python3
# author: Hans D. Escobar. H - e-mail: escobar.hans@gmail.com

import os
import sys
import numpy as np
import pandas as pd
from scipy.stats import norm
from scipy.stats import t
sys.path.insert(0, os.path.abspath('..'))
from popecology import abundance_estimation_population_models as aspm
from popecology.abundance_estimation_methods import LincolnPetersen

## Asunciones minimas de estos métodos

1. Población cerrada. La abundancia (número total de animales) $N$ no cambia durante el tiempo de estudio.
2. No existe perdida de marcas.
3. La probabilidad de captura $\hat{p}$ es __constante__:
    - en el tiempo (e.g. meterología, esfuerzo de captura).
    - en el espacio (e.g. hábitat)
    - entre los individuos (sexo, edad, experiencia previa).

In [10]:
single_simulation_parameters = {
    "initial_size" : 1000,
    "capture_distribution" : (0.2, 0.2),
    "death_distribution" : (0, 0),
    "inmigration_rate" : 0,
    "mark_lost_probability" : 0}

### Lincoln-Petersen

Los supuestos son los 3 generales, con las siguientes precisiones:

- Para todo individuo $x_i$, $P(\text{captura}\:|\:x_i) = \hat{p}$
- $P(\text{captura}\:|\:\text{previamente capturado}) = P(\text{captura}\:|\:\text{nunca capturado})$
- $\hat{p}_1$ puede ser diferente de $\hat{p}_2$


Para este método tenemos 3 estimadores:
    - Simple (sesgado)
    - Corregido de Bailey (insesgado)
    - Corregido de Chapman (insesgado)

Por definición, un estimador insesgado es aquel estadistico cuyo valor esperado es igual al parametro de la población. Por teorema de los números grandes, a medida que el numero de muestras tiende a infinito este se acerca al valor esperado, vamos a emplearlo para verificar esta propiedad.

In [11]:
iteration_number = 10000
true_population_size = 500
single_simulation_parameters = {
    "initial_size" : true_population_size,
    "capture_distribution" : (0.082, 0.082),
    "death_distribution" : (0, 0),
    "inmigration_rate" : 0,
    "mark_lost_probability" : 0}



In [12]:
simulation_output = {
    "simple_estimator": list(),
    "chapman_estimator": list(),
    "chapman_sd_error": list(),
    "bailey_estimator": list(),
    "bailey_sd_error": list(),
}

for trial in range(iteration_number):
    current_simulation = aspm.CmrPopulation( **single_simulation_parameters)
    captured = current_simulation.sample_and_mark(100)[current_simulation.unmarked_id]
    recaptured_unmarked, recaptured_marked = current_simulation.sample_but_not_mark(100).values()
    param = [captured, recaptured_unmarked, recaptured_marked]
    simulation_output["simple_estimator"].append(LincolnPetersen.simple_biased_statistic(*param))
    chapman = list(LincolnPetersen.chapman_unbiased_summary(*param).values())
    simulation_output["chapman_estimator"].append(chapman[0])
    simulation_output["chapman_sd_error"].append(chapman[1])
    bailey = list(LincolnPetersen.bailey_unbiased_summary(*param).values())
    simulation_output["bailey_estimator"].append(bailey[0])
    simulation_output["bailey_sd_error"].append(bailey[1])


In [13]:
simulation_df = pd.DataFrame.from_dict(simulation_output)
number_of_samples_without_recaptured_marked =len(simulation_df[simulation_df.simple_estimator == 0])
# Remove samples without recaptured marked individuals
simulation_df = simulation_df[simulation_df.simple_estimator != 0]


In [14]:
abs(simulation_df[["simple_estimator", "chapman_estimator", "bailey_estimator"]].mean() - true_population_size)

simple_estimator     421.325697
chapman_estimator    422.780447
bailey_estimator     430.349182
dtype: float64

In [15]:
current_simulation = aspm.CmrPopulation( **single_simulation_parameters)
captured = current_simulation.sample_and_mark(100)[current_simulation.unmarked_id]
recaptured_unmarked, recaptured_marked = current_simulation.sample_but_not_mark(100).values()
[captured, recaptured_unmarked, recaptured_marked]

[9, 7, 0]

In [16]:
current_simulation = aspm.CmrPopulation(
        **single_simulation_parameters
    )
current_simulation.sample_and_mark(10)

{'unmarked': 0, 'marked': 0}

In [17]:
current_simulation.sample_record

[{'unmarked': 0, 'marked': 0}, {'unmarked': 0, 'marked': 0}]

In [18]:
current_simulation.population_record

[{'unmarked': 500, 'marked': 0}, {'unmarked': 500, 'marked': 0}]

In [19]:
current_simulation.sample_but_not_mark(10)


{'unmarked': 0, 'marked': 0}

Cuál es el efecto de:

a. Atracción por la trampa y/o cebo.

> aumenta la P(captura)

b. Acostumbramiento a la trampa.

> P(captura | marcado) >   P(captura | no-marcado)

c. Diferencias en técnicas de muestreo: tamaño de trampa, tipo de trampa

> Altera la P(captura)

d. Pérdida de las marcas.

> P(perdida marca)

e. Efecto de la marca en la sobrevivencia de los individuos

> P(muerte | marcado) vs. P(muerte | no-marcado)

f. Efecto de la cacería en las estimaciones de tamaño poblacional

> P(muerte) > 0